In [54]:
# Script to download the ERA5 dataset from WeatherBench2. The script filters the data to the Europe grid and the specified date range.
import apache_beam 
import weatherbench2
import xarray as xr
import gcsfs
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [60]:

# Set parameters
era5_path = 'gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr'
date_range = pd.date_range(f"2010-01-01", f"2022-12-31T18", freq = "6h")
# # Europe grid
lat_range = np.arange(35, 75, 0.25)
lon_range = np.append(np.arange(347.5,360, 0.25),np.arange(0, 42.5,0.25))
variables = ["2m_temperature", "10m_u_component_of_wind", "10m_v_component_of_wind", "temperature", "geopotential", "land_sea_mask", "geopotential_at_surface"]
output_dir = "../era5/"
filename = "era5.nc"
# Create output dir
os.makedirs(output_dir, exist_ok = True)

# Open file from weatherbench and filter
data = xr.open_zarr(era5_path)
data_reduced = data[variables].sel(latitude = lat_range, longitude = lon_range, method = "nearest").sel(time = date_range, method = "nearest")
data_reduced["geopotential"] = data_reduced["geopotential"].sel(level = 500)
data_reduced["temperature"] = data_reduced["temperature"].sel(level = 850)
# Reindex time
#data_reduced["time"] = xr.coding.times.CFDatetimeCoder().encode(data_reduced.time)
# Rechunk and save
data_reduced.chunk("auto")#.to_netcdf(output_dir + filename)


<xarray.Dataset> Size: 13GB
Dimensions:                  (time: 18992, latitude: 160, longitude: 220,
                              level: 13)
Coordinates:
  * latitude                 (latitude) float32 640B 35.0 35.25 ... 74.5 74.75
  * level                    (level) int64 104B 50 100 150 200 ... 850 925 1000
  * longitude                (longitude) float32 880B 347.5 347.8 ... 42.0 42.25
  * time                     (time) datetime64[ns] 152kB 2010-01-01 ... 2022-...
Data variables:
    2m_temperature           (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    10m_u_component_of_wind  (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    temperature              (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    geopotential             (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    land_sea_mask            (latitude, longitude) float32 141kB dask.array<chunksize=(160, 220), meta=np.ndarray>
    geopotential_at_surface  (latitude, longitude) float32 141kB dask.array<chunksize=(160, 220), meta=np.ndarray>

In [70]:
# HRES
ifs_ens_path = 'gs://weatherbench2/datasets/ifs_ens/2018-2022-1440x721_mean.zarr'

date_range = pd.date_range(f"2018-01-01", f"2022-12-31T06", freq = "24h")
variables = ["2m_temperature", "10m_u_component_of_wind", "10m_v_component_of_wind", "temperature", "geopotential"]

# Open file from weatherbench and filter (12h prediction horizon)
data = xr.open_zarr(ifs_ens_path)
data_reduced = data[variables].sel(latitude = lat_range, longitude = lon_range, method = "nearest").isel(prediction_timedelta = 2)
data_reduced["geopotential"] = data_reduced["geopotential"].sel(level = 500)
data_reduced["temperature"] = data_reduced["temperature"].sel(level = 850)
# Rechunk and save
data_reduced.chunk("auto")#.to_netcdf(output_dir + filename)

/tmp/ipykernel_594363/1284366429.py:8: FutureWarning: In a future version, xarray will not decode timedelta values based on the presence of a timedelta-like units attribute by default. Instead it will rely on the presence of a timedelta64 dtype attribute, which is now xarray's default way of encoding timedelta64 values. To continue decoding timedeltas based on the presence of a timedelta-like units attribute, users will need to explicitly opt-in by passing True or CFTimedeltaCoder(decode_via_units=True) to decode_timedelta. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  data = xr.open_zarr(ifs_ens_path)


<xarray.Dataset> Size: 3GB
Dimensions:                  (time: 3652, latitude: 160, longitude: 220,
                              level: 3)
Coordinates:
  * latitude                 (latitude) float32 640B 35.0 35.25 ... 74.5 74.75
  * level                    (level) int32 12B 500 700 850
  * longitude                (longitude) float32 880B 347.5 347.8 ... 42.0 42.25
    prediction_timedelta     timedelta64[ns] 8B 12:00:00
  * time                     (time) datetime64[ns] 29kB 2018-01-01 ... 2022-1...
Data variables:
    2m_temperature           (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    10m_u_component_of_wind  (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    temperature              (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    geopotential             (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>

In [79]:
data_reduced.drop_vars(["level", "prediction_timedelta"])

<xarray.Dataset> Size: 3GB
Dimensions:                  (time: 3652, latitude: 160, longitude: 220)
Coordinates:
  * latitude                 (latitude) float32 640B 35.0 35.25 ... 74.5 74.75
  * longitude                (longitude) float32 880B 347.5 347.8 ... 42.0 42.25
  * time                     (time) datetime64[ns] 29kB 2018-01-01 ... 2022-1...
Data variables:
    2m_temperature           (time, latitude, longitude) float32 514MB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    10m_u_component_of_wind  (time, latitude, longitude) float32 514MB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 514MB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    temperature              (time, latitude, longitude) float32 514MB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    geopotential             (time, latitude, longitude) float32 514MB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>

In [82]:
xr.open_zarr("/home/groups/ai/datasets/weather_forecasting/ifs_ens.zarr", consolidated=False)

<xarray.Dataset> Size: 3GB
Dimensions:                  (time: 3652, latitude: 160, longitude: 220)
Coordinates:
  * time                     (time) datetime64[ns] 29kB 2018-01-01 ... 2022-1...
  * longitude                (longitude) float32 880B 347.5 347.8 ... 42.0 42.25
  * latitude                 (latitude) float32 640B 35.0 35.25 ... 74.5 74.75
Data variables:
    2m_temperature           (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    geopotential             (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    10m_u_component_of_wind  (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    temperature              (time, latitude, longitude) float32 514MB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>

# Load finished data

In [26]:
dataset = xr.open_zarr("/home/groups/ai/datasets/weather_forecasting/era5.zarr", consolidated=False)
dataset

<xarray.Dataset> Size: 13GB
Dimensions:                  (time: 18992, latitude: 160, longitude: 220,
                              level: 13)
Coordinates:
  * longitude                (longitude) float32 880B 347.5 347.8 ... 42.0 42.25
  * level                    (level) int64 104B 50 100 150 200 ... 850 925 1000
  * latitude                 (latitude) float32 640B 35.0 35.25 ... 74.5 74.75
  * time                     (time) datetime64[ns] 152kB 2010-01-01 ... 2022-...
Data variables:
    10m_u_component_of_wind  (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    temperature              (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    land_sea_mask            (latitude, longitude) float32 141kB dask.array<chunksize=(160, 220), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    2m_temperature           (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    geopotential_at_surface  (latitude, longitude) float32 141kB dask.array<chunksize=(160, 220), meta=np.ndarray>
    geopotential             (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>

## Create global mean and stds statistics

In [27]:
WB_INPUT = ["2m_temperature","temperature", "10m_u_component_of_wind", "10m_v_component_of_wind", "geopotential", "land_sea_mask", "geopotential_at_surface"]

In [28]:
statistics = np.zeros((len(WB_INPUT), 2))
for i,var in enumerate(WB_INPUT):
    if var == "land_sea_mask":
        statistics[i,0] = 0
        statistics[i,1] = 1
    else:
        statistics[i,0] = dataset[var].mean().compute()
        statistics[i,1] = dataset[var].std().compute()

np.save("/home/groups/ai/datasets/weather_forecasting/statistics.npy", statistics)

In [32]:
test = np.load("/home/groups/ai/datasets/weather_forecasting/era5_statistics.npy")

In [33]:
mu, sigma = np.split(test, 2, axis = -1)
mu.shape, sigma.shape

((7, 1), (7, 1))

# Dataset

In [44]:
dataset = xr.open_zarr("/home/groups/ai/datasets/weather_forecasting/era5.zarr", consolidated=False)[WB_INPUT]
dataset

<xarray.Dataset> Size: 13GB
Dimensions:                  (time: 18992, latitude: 160, longitude: 220)
Coordinates:
  * longitude                (longitude) float32 880B 347.5 347.8 ... 42.0 42.25
  * latitude                 (latitude) float32 640B 35.0 35.25 ... 74.5 74.75
  * time                     (time) datetime64[ns] 152kB 2011-01-01 ... 2023-...
Data variables:
    2m_temperature           (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    temperature              (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    10m_u_component_of_wind  (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    10m_v_component_of_wind  (time, latitude, longitude) float32 3GB dask.array<chunksize=(1, 160, 220), meta=np.ndarray>
    geopotential             (time, latitude, longitude) float32 3GB dask.array<chunksize=(953, 160, 220), meta=np.ndarray>
    land_sea_mask            (latitude, longitude) float32 141kB dask.array<chunksize=(160, 220), meta=np.ndarray>
    geopotential_at_surface  (latitude, longitude) float32 141kB dask.array<chunksize=(160, 220), meta=np.ndarray>

In [98]:
dataset.sizes["longitude"]

220

In [86]:
len(dataset.latitude.values)

160

In [45]:
t1 = dataset.sel(time = slice("2018", "2020")).isel(time = slice(3,6)).to_array()

In [66]:
input = t1.isel(time = slice(0,2))
target = t1.isel(time = 2, variable = 0)

In [68]:
target

<xarray.DataArray (latitude: 160, longitude: 220)> Size: 141kB
dask.array<getitem, shape=(160, 220), dtype=float32, chunksize=(160, 220), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 880B 347.5 347.8 348.0 ... 41.75 42.0 42.25
  * latitude   (latitude) float32 640B 35.0 35.25 35.5 ... 74.25 74.5 74.75
    time       datetime64[ns] 8B 2018-01-02T06:00:00
    variable   <U14 56B '2m_temperature'

In [67]:
input

<xarray.DataArray (variable: 7, time: 2, latitude: 160, longitude: 220)> Size: 2MB
dask.array<getitem, shape=(7, 2, 160, 220), dtype=float32, chunksize=(1, 1, 160, 220), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 880B 347.5 347.8 348.0 ... 41.75 42.0 42.25
  * latitude   (latitude) float32 640B 35.0 35.25 35.5 ... 74.25 74.5 74.75
  * time       (time) datetime64[ns] 16B 2018-01-01T18:00:00 2018-01-02
  * variable   (variable) object 56B '2m_temperature' ... 'geopotential_at_su...

In [70]:
input.isel(time = -1, variable = 0)

<xarray.DataArray (latitude: 160, longitude: 220)> Size: 141kB
dask.array<getitem, shape=(160, 220), dtype=float32, chunksize=(160, 220), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 880B 347.5 347.8 348.0 ... 41.75 42.0 42.25
  * latitude   (latitude) float32 640B 35.0 35.25 35.5 ... 74.25 74.5 74.75
    time       datetime64[ns] 8B 2018-01-02
    variable   <U14 56B '2m_temperature'

In [69]:
target - input.isel(time = -1, variable = 0)

<xarray.DataArray (latitude: 160, longitude: 220)> Size: 141kB
dask.array<sub, shape=(160, 220), dtype=float32, chunksize=(160, 220), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float32 880B 347.5 347.8 348.0 ... 41.75 42.0 42.25
  * latitude   (latitude) float32 640B 35.0 35.25 35.5 ... 74.25 74.5 74.75
    variable   <U14 56B '2m_temperature'